In [14]:
# codes parsed from original DaXi controller codes
"""making ao channel to send a sequence of signals"""
import nidaqmx
import nidaqmx.system
import numpy as np
import time 

# Define tools


In [45]:
# create a task to set a device to "go home", meaning the neutral position with everything centered.
def gohome(params):
    if params['verbose']:
        print('Now setting ['+params['controlled device name']+'] to the '+
              'home position [voltage = '+str(params['home voltage'])+'v] with ' +
              '[offset = '+str(params['home offset voltage'])+'v,'+str(params['home offset option'])+'] '+
              'through channel [\''+params['channel_string']+'\']')
    if params['channel I/O type'] == "AO":
        task = nidaqmx.Task(params['controlled device name'] )
        task.ao_channels.add_ao_voltage_chan(params['channel_string'])
        params['task']=task
        time.sleep(1)  # for manual inspection to make sure it took action.
        try:
            if params['home offset option'] is True:
                value = params['home voltage'] + params['home offset voltage']
            else:
                value = params['home voltage']

            task.write(value,auto_start=True)
            if params['verbose']: print('Task perfromed successfully.')
        except:
            if params['verbose']: print('Failed performing the task.')
        task.close()
    else:
        if params['verbose']: print('only DAQ AO channel is supported currently')
    if params['verbose']:
        print('Task closed.')

In [46]:
def gohome_all(paramslist):
    # create an AO task
    task = nidaqmx.Task()
    values=[]
    # go through allt he pramslist and (1) add AO channels and (2) prepare write values list.
    for params in paramslist:
        if params['verbose']:
            print('Now setting ['+params['controlled device name']+'] to the '+
                  'home position [voltage = '+str(params['home voltage'])+'v] with ' +
                  '[offset = '+str(params['home offset voltage'])+'v,'+str(params['home offset option'])+'] '+
                  'through channel [\''+params['channel_string']+'\']')
            
        # add all the AO channels to the AO tasks 
        if params['channel I/O type'] == "AO":
            task.ao_channels.add_ao_voltage_chan(params['channel_string'])
            params['task'] = task
            
            # generate the value for this AO channel
            try:
                if params['home offset option'] is True:
                    value = params['home voltage'] + params['home offset voltage']
                else:
                    value = params['home voltage']
                    
                # append the value to the value list
                values.append(value)
                if params['verbose']: print('Task performed successfully.')
            except:
                if params['verbose']: print('Failed performing the task.')
            
        else:
            if params['verbose']: print('only DAQ AO channel is supported currently')
            
    task.write(values,auto_start=True)
    return task

# Below is trying to pull out the channel type from the device channel string

# think about which way is better
try "launch - go" framework and see if it feels right.

Launch different channels onto the same task.

In [16]:
# get the local nidaqmx systems.
system = nidaqmx.system.System.local()
cstr = "cDAQ1AO/ao0" # provide a channel string name
cstr2 = "/cDAQ1/Ctr0InternalOutput"

In [17]:
d = system.devices[cstr2]

In [18]:
d.name

'/cDAQ1/Ctr0InternalOutput'

In [19]:
l=nidaqmx.system.physical_channel.PhysicalChannel(cstr)

In [20]:
l.name

'cDAQ1AO/ao0'

In [21]:
# list the names of all the DAQ cards:
system.devices.device_names

['cDAQ1', 'cDAQ1-AO-NI9263', 'cDAQ2-DIO-NI9401', 'cDAQ3-AO-NI9263', 'SimDev1']

## Want to find out all available channels.

In [22]:
# check properties of the cDAQ1DIO card:
d = system.devices['cDAQ2-DIO-NI9401']

# counter has 80 MHz rate:
d.ci_max_timebase

d.ci_physical_chans.channel_names

['cDAQ2-DIO-NI9401/ctr0',
 'cDAQ2-DIO-NI9401/ctr1',
 'cDAQ2-DIO-NI9401/ctr2',
 'cDAQ2-DIO-NI9401/ctr3']

In [23]:
# check the cDAQ1 card
d1 = system.devices['cDAQ2-DIO-NI9401']

# counter size is 32 bits
print(d1.ci_max_size)

# counter max time base is 80 MHz
print(d1.ci_max_timebase)




32
80000000.0


# Scratch board below  -  2022-08-31 - configure daq ports for the new daxi

## Define tools

In [24]:
import nidaqmx
import nidaqmx.system
import numpy as np
import time
from time import sleep

# define parameters
# in this cell we store the final voltages.
SG={} # scanning galvo
SG['instrument name'] = "DaXi"
SG['channel I/O type'] = "AO"  #
SG['channel_string'] = "cDAQ3-AO-NI9263/ao0"  # name of the DAQ channel that will be used to control the device
SG['controlled device name'] = "scanning galvo"  # name of the deviced to be controlled.
SG['description'] = "at C-BFP, to scan the position of the light sheet."  # purpose of this device.
SG['home voltage'] = 0  # voltage of the home value to be sent to the specified DAQ channel.
SG['home offset voltage'] = 0  # this is an offset voltage to set on top of the home voltage value.
SG['home offset option'] = True  # the offset will be applied when this option is True.
SG['verbose'] = False  # when True, status messages will be printed.

# O1
O1={} # scanning galvo
O1['instrument name'] = "DaXi"
O1['channel I/O type'] = "AO"  #
O1['channel_string'] = "cDAQ3-AO-NI9263/ao1"  # name of the DAQ channel that will be used to control the device
O1['controlled device name'] = "o1"  # name of the deviced to be controlled.
O1['description'] = "set O1 offset."  # purpose of this device.
O1['home voltage'] = 0  # voltage of the home value to be sent to the specified DAQ channel.
O1['home offset voltage'] = 0  # this is an offset voltage to set on top of the home voltage value.
O1['home offset option'] = True  # the offset will be applied when this option is True.
O1['verbose'] = False  # when True, status messages will be printed.

# define parameters
gamma_G={} # gamma galvo
gamma_G['instrument name'] = "DaXi"
gamma_G['channel I/O type'] = "AO"  #
gamma_G['channel_string'] = "cDAQ1-AO-NI9263/ao1"  # name of the DAQ channel that will be used to control the device
gamma_G['controlled device name'] = "gamma galvo"  # name of the deviced to be controlled.
gamma_G['description'] = " at C-FP, introduce in-plane tilts in the light sheet, used for strip reduction"  # purpose of this device.
gamma_G['home voltage'] = 0  # voltage of the home value to be sent to the specified DAQ channel.
gamma_G['home offset voltage'] = 0 # this is an offset voltage to set on top of the home voltage value.
gamma_G['home offset option'] = True  # the offset will be applied when this option is True.
gamma_G['verbose'] = False  # when True, status messages will be printed.

# define parameters
beta_G={} # beta galvo
beta_G['instrument name'] = "DaXi"
beta_G['channel I/O type'] = "AO"  #
beta_G['channel_string'] = "cDAQ1-AO-NI9263/ao0"  # name of the DAQ channel that will be used to control the device
beta_G['controlled device name'] = "beta galvo"  # name of the deviced to be controlled.
beta_G['description'] = "at C-FP, introduce out-of-plane tilt in the  light sheet, used for changing pitch angle"  # purpose of this device.
beta_G['home voltage'] = 0  # voltage of the home value to be sent to the specified DAQ channel.
beta_G['home offset voltage'] = 0  # this is an offset voltage to set on top of the home voltage value.
beta_G['home offset option'] = True  # the offset will be applied when this option is True.
beta_G['verbose'] = False  # when True, status messages will be printed.

# define parameters
VSG1={} # view switching galvo 1
VSG1['instrument name'] = "DaXi"
VSG1['channel I/O type'] = "AO"  #
VSG1['channel_string'] = "cDAQ1-AO-NI9263/ao2"  # name of the DAQ channel that will be used to control the device
VSG1['controlled device name'] = "view switching galvo 1 (closer to O1)"  # name of the deviced to be controlled.
VSG1['description'] = "not in C planes, used for switching the view."  # purpose of this device.
VSG1['home voltage'] = 0  # voltage of the home value to be sent to the specified DAQ channel.
VSG1['home offset voltage'] = 0  # this is an offset voltage to set on top of the home voltage value.
VSG1['home offset option'] = True  # the offset will be applied when this option is True.
VSG1['verbose'] = False  # when True, status messages will be printed.

# define parameters
VSG2={} # view switching galvo 2
VSG2['instrument name'] = "DaXi"
VSG2['channel I/O type'] = "AO"  #
VSG2['channel_string'] = "cDAQ1-AO-NI9263/ao2"  # name of the DAQ channel that will be used to control the device
VSG2['controlled device name'] = "view switching galvo 2 (closer to O2)"  # name of the deviced to be controlled.
VSG2['description'] = "not in C planes, used for switching the view."  # purpose of this device.
VSG2['home voltage'] = 0  # voltage of the home value to be sent to the specified DAQ channel.
VSG2['home offset voltage'] = 0  # this is an offset voltage to set on top of the home voltage value.
VSG2['home offset option'] = True  # the offset will be applied when this option is True.
VSG2['verbose'] = False  # when True, status messages will be printed.


In [53]:
beta_G

C:\Users\Chan Zuckerberg\.conda\envs\sandbox1\lib\site-packages\nidaqmx\task.py:102: ResourceWarning: Task of name "_unnamedTask<A>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(
C:\Users\Chan Zuckerberg\.conda\envs\sandbox1\lib\site-packages\nidaqmx\task.py:102: ResourceWarning: Task of name "_unnamedTask<B>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(
C:\Users\Chan Zuckerberg\.conda\envs\sandbox1\lib\site-packages\nidaqmx\task.py:102: ResourceWarning: Task of name "_unnamedTask<C>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(
C:\Users\Chan Zuckerberg\.conda\envs\sandbox1\lib\site-packages\nidaqmx\task.py:102: ResourceWarning: Task of name "_unnamedTask<0>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
 

DaqError: Task specified is invalid or does not exist.
Status Code: -200088

# adjusting voltages for beta and gamma galvos.

In [51]:
beta_G['home voltage']=0
gamma_G['home voltage']=0

t=gohome_all([gamma_G, beta_G])

In [52]:
t.close()

# Adjusting VS galvos for view with 1 mirror

In [ ]:
# view with 1 mirror.
sleep(2)
VSG1['home offset voltage']= -4.28# -4.06 negative moves left.
VSG2['home offset voltage']= 4.65
gohome(VSG1)
gohome(VSG2)

# Adjust VS galvo for view with 2 mirrors

In [53]:
sleep(2)
VSG1['home offset voltage'] = 4.125 #4.255 #4.21
VSG2['home offset voltage'] = -3.525
gohome(VSG1) 
gohome(VSG2)

# Adjusting gamma and beta galvos

In [52]:
sleep(1)
gamma_G['home offset voltage'] = 0.18    #-1.1 # (negative moves down)
gohome(gamma_G)
beta_G['home offset voltage'] = 0.4    #1.7 #0.84#0.87 # ( negative moves to right)
gohome(beta_G)       

# Adjut for beta/gamma galvo conjugation.

In [47]:
gamma_G['home offset voltage'] = 0 # (negative moves down)
gohome(gamma_G)

In [190]:
for _ in np.arange(3):
    beta_G['home offset voltage'] = 1.3 # ( negative moves to right)
    gohome(beta_G)
    go    beta_G['home offset voltage'] = 0.5 # ( negative moves to right) 
home(beta_G)

# Adjusting SG galvos

In [28]:
sleep(0.5)
SG['home offset voltage'] = 1.57 #1.65 
gohome(SG    

# Adjust O1 position for calibration

In [581]:
O1['home voltage'] = 5 # (negative moves down)
gohome(O1)

In [ ]:
O1['home voltage'] = -5 # (negative moves down)
gohome(O1)

In [656]:
O1['home voltage'] = 0 # (negative moves down)
gohome(O1)

In [ ]:
# adjust galvo for 1 mirror
# view with 1 mirror.
VSG2['home offset voltage']= 4
gohome(VSG2)
VSG1['home offset voltage']= 0# -4.06
gohome(VSG1)

for dv in [-4.17,-7.]:
    VSG1['home offset voltage']= -4.17# -4.06
    gohome(VSG1)


In [29]:
for _ in np.arange(100):
    SG['home voltage'] = 0
    gohome(SG)
    gohome(SG)
    SG['home voltage'] = 0.2
    gohome(SG)
    SG['home voltage'] = 2


    

KeyboardInterrupt: 

In [30]:
for i in np.arange(0,2,0.01):
    SG['home voltage'] = i
    gohome(SG)


In [11]:
scanstep=400
convert_ratio=159
scanstep/2/convert_ratio*2


2.5157232704402515

#  Generate periodic control signal
## Needs:
1. timer
2. counter
3. define rates for the periodic control sequence.
4. define # of samples within a period.

In [6]:
# try:
# define an ao task to generate sin wave output.

# make the trigger of this ao task to be a pulse train, which is the output of a counter

# define a sampler, make it generate a pulse train, and use it as a trigger for the ao task.

# because daq cards only allow for 1 task per type for AI/O, 
# DI/O tasks if hardware timing is used. so these functions only "configures" the tasks, 
# they dont create and close tasks.

def launch_sampler(param, task):
    # try to make a pulse train as the sampling function for the ao
    task.co_channels.add_co_pulse_chan_freq(param['channel_string_I'],
                                                       idle_state=nidaqmx.constants.Level.LOW,
                                                       freq=param['sampling_rate'])
    if sampler['sample_mode'] == 'Finite':
        m=nidaqmx.constants.AcquisitionType.FINITE

    if sampler['sample_mode'] == 'Continuous':
        m=nidaqmx.constants.AcquisitionType.CONTINUOUS

    task.timing.cfg_implicit_timing(sample_mode=m,
                                    samps_per_chan=param['num_sample'])

    task.triggers.start_trigger.cfg_dig_edge_start_trig(trigger_source = param['trigger_source'],
                                                        trigger_edge = nidaqmx.constants.Slope.RISING)

    task.triggers.start_trigger.retriggerable = True
    
# and make a switch method [daq-tools].
def launch_switch(param, task, status='on'):
    "it has to be a digital channel."
    task.do_channels.add_do_chan(param['channel_string'])
    if status == 'on':
        task.write([True], auto_start=True)
    else:
        task.write([False], auto_start=True)

def reset_switch(switch_task, status='on'):
    if status == 'on':
        task.write([False], auto_start=True)
        task.write([True], auto_start=True)
    else:
        task.write([True], auto_start=True)
        task.write([False], auto_start=True)


# Prepare parameters

In [43]:
# define parameters of a swtich to be used as a trigger. (interface.)
# define parameters
sample_n=300
x=np.linspace(0,np.pi*2,sample_n*5)
y=np.sin(x)

sw = {} # switcher parameters
sw['instrument_name'] = "DaXi"
sw['channel I/O type'] = "DO"  #
sw['channel_string'] = "cDAQ1DIO/port0/line3"  # name of the DAQ channel that will be used to control the device, choose one.
sw['controlled_device_name'] = "switch"  # name of the deviced to be controlled. (like a switch button.)
sw['purpose'] = "this serves as a power switch to turn something on/off"  # purpose of this device.
sw['verbose'] = True  # when True, status messages will be printed.

# define sampler parameters that generates a pulsed train from an internal counter.
sampler={}
sampler['instrument_name'] = "DaXi"
sampler['channel I/O type'] = "Counter"  
sampler['channel_string_I'] = "cDAQ1/_ctr0"  
sampler['channel_string_O'] = "/cDAQ1/Ctr0InternalOutput"  
sampler['controlled_device_name'] = "sampler"  
sampler['purpose'] = "to configure a counter to generate a pulse train" 
sampler['verbose'] = True  
sampler['sampling_rate'] = sample_n
sampler['sample_mode'] = 'Finite' # specify Finite or Continuous
sampler['num_sample'] = sample_n
sampler['trigger_source'] = "/cDAQ1/PFI0" #sw['channel_string']

# define ao scan parameters 
ao_scan = {}
ao_scan['instrument_name'] = "DaXi"
ao_scan['channel I/O type'] = "AO"
ao_scan['sampler_channel_str'] = sampler['channel_string_O']
ao_scan['controlled_device_name'] = "a scanner"
ao_scan['ao_profile'] = list(y)

def get_vtrain_sine(sample_n):
    x=np.linspace(0,np.pi*2,sample_n)
    y=np.sine(x)



# Pre-launch tasks

In [10]:
sampler_task = nidaqmx.Task()
launch_sampler(sampler, task=sampler_task)

switch_task=nidaqmx.Task()
launch_switch(status='on', param=sw, task=switch_task)

# Launch tasks

In [11]:
sampler_task.start()

# Trigger tasks


In [27]:
# setup the scanners.

reset_switch(status='on', switch_task=switch_task)

C:\Users\PiscesScope\.conda\envs\coPylot\lib\site-packages\nidaqmx\task.py:102: ResourceWarning: Task of name "_unnamedTask<6>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(


DaqError: Specified channel cannot be added to the task, because a channel with the same name is already in the task.
Channel Name: cDAQ1DIO/port0/line3

Task Name: _unnamedTask<2>

Status Code: -200489

# Close tasks

In [12]:
sampler_task.close()
switch_task.close()

In [31]:
reset_switch(status='off', switch_task=switch_task)

In [8]:
action_params=ao_scan
device_params=SG

task_ao_scanner = nidaqmx.Task()
task_ao_scanner.ao_channels.add_ao_voltage_chan(device_params['channel_string'])  # scanning galvo
task_ao_scanner.timing.cfg_samp_clk_timing(rate=sample_n,
                                   source=action_params['sampler_channel_str'],
                                   sample_mode=nidaqmx.constants.AcquisitionType.CONTINUOUS)
task_ao_scanner.write(action_params['ao_profile'])
task_ao_scanner.start()

In [15]:
task_ao_scanner.stop()
task_ao_scanner.close()

In [48]:
gohome(SG)

Now setting [scanning galvo] to the home position [voltage = 0v] with [offset = 0v,False] through channel ['cDAQ1AO/ao0']
Task perfromed successfully.
Task closed.


In [ ]:
# setup a counter task to pair with the controller task.
#? what does counter do?
counter = nidaqmx.Task("counter0")
counter.close()

In [51]:
counter.close()
task.close()

C:\Users\PiscesScope\.conda\envs\coPylot\lib\site-packages\nidaqmx\task.py:457: ResourceWarning: Attempted to close NI-DAQmx task of name "counter0" but task was already closed.
  warnings.warn(


In [ ]:
    with nidaqmx.Task() as task:
        task.ao_channels.add_ao_voltage_chan(ch)
        task.write([value], auto_start=True)

In [62]:
time.sleep(2)

In [38]:
def view_switching_galvo1(channel="cDAQ1AO/ao1", voltage=4.2):
    # create task to set the channel to the desired voltage.
    
    
    
    

32

In [18]:
counter_names = [ci.name for ci in DAQ_device.ci_physical_chans]
print(counter_names)

['cDAQ1DIO/ctr0', 'cDAQ1DIO/ctr1', 'cDAQ1DIO/ctr2', 'cDAQ1DIO/ctr3']


In [19]:
print([co.name for co in DAQ_device.co_physical_chans])

['cDAQ1DIO/ctr0', 'cDAQ1DIO/ctr1', 'cDAQ1DIO/ctr2', 'cDAQ1DIO/ctr3', 'cDAQ1DIO/freqout']


In [22]:
DAQ_device.co_max_size

32

In [26]:
DAQ_device.di_lines.channel_names

['cDAQ1DIO/port0/line0',
 'cDAQ1DIO/port0/line1',
 'cDAQ1DIO/port0/line2',
 'cDAQ1DIO/port0/line3',
 'cDAQ1DIO/port0/line4',
 'cDAQ1DIO/port0/line5',
 'cDAQ1DIO/port0/line6',
 'cDAQ1DIO/port0/line7']

In [27]:
DAQ_device.do_lines.channel_names

['cDAQ1DIO/port0/line0',
 'cDAQ1DIO/port0/line1',
 'cDAQ1DIO/port0/line2',
 'cDAQ1DIO/port0/line3',
 'cDAQ1DIO/port0/line4',
 'cDAQ1DIO/port0/line5',
 'cDAQ1DIO/port0/line6',
 'cDAQ1DIO/port0/line7']

In [28]:
DAQ_device.name

'cDAQ1DIO'